In [1]:
import pandas as pd

In [ ]:
#Load Original datasets
df_08 = pd.read_excel('all_alpha_08.xls')
df_18 = pd.read_excel('all_alpha_18.xlsx')

In [2]:
#Load the new datasets
df_08 = pd.read_csv('data_08.csv')
df_18 = pd.read_csv('data_18.csv')

In [ ]:
# view dimensions of dataset
df_08.shape

In [ ]:
# view dimensions of dataset
df_18.shape

In [ ]:
df_08.head()

In [ ]:
df_18.head()

In [ ]:
df_08.info()

In [ ]:
#No. of duplicate drows
sum(df_08.duplicated())

In [ ]:
#No of rows with missing Data
2404 - 2205

In [ ]:
df_18.info()

In [ ]:
df_08['SmartWay'].nunique()

In [ ]:
df_18['SmartWay'].nunique()

In [ ]:
df_18['Cert Region'].nunique()

In [ ]:
df_08['Sales Area'].nunique()

In [ ]:
df_08['Trans'].nunique()

In [ ]:
df_08['Cyl'].nunique()

In [ ]:
df_18['Cyl'].nunique()

In [ ]:
df_08['Fuel'].value_counts()

In [ ]:
df_18['Fuel'].value_counts()

In [ ]:
df_08.drop(columns=['Stnd', 'Underhood ID', 'FE Calc Appr', 'Unadj Cmb MPG'], inplace = True)

In [ ]:
df_18.drop(columns= ['Stnd', 'Stnd Description', 'Underhood ID', 'Comb CO2'], inplace = True)

In [ ]:
df = pd.DataFrame({"A": [1, 2, 3], "B": [4, 5, 6]})
print(df)

In [ ]:
# rename Sales Area to Cert Region
df_08.rename( columns={"sales_area": "cert_region"},inplace = True)
df_08.head(2)

In [ ]:
# replace spaces with underscores and lowercase labels for 2008 dataset
df_08.rename(columns=lambda x: x.strip().lower().replace(" ", "_"), inplace=True)

# confirm changes
df_08.head(1)

In [ ]:
# replace spaces with underscores and lowercase labels for 2018 dataset

df_18.rename(columns=lambda x: x.strip().lower().replace(" ", "_"), inplace=True)
# confirm changes
df_18.head(1)

In [ ]:
# confirm column labels for 2008 and 2018 datasets are identical
df_08.columns == df_18.columns

In [ ]:
# make sure they're all identical like this
(df_08.columns == df_18.columns).all()

In [ ]:
# save new datasets for next section
df_08.to_csv('data_08.csv', index=False)
df_18.to_csv('data_18.csv', index=False)

In [ ]:
#Load the new datasets
df_08 = pd.read_csv('data_08.csv')
df_18 = pd.read_csv('data_18.csv')

In [ ]:
df_08.head()

In [ ]:
# check value counts for the 2008 cyl column
df_08['cyl'].value_counts()

In [ ]:
# Extract int from strings in the 2008 cyl column
df_08['cyl'] = df_08['cyl'].str.extract('(\d+)').astype(int)

In [ ]:
# Check value counts for 2008 cyl column again to confirm the change
df_08['cyl'].value_counts()

In [ ]:
# convert 2018 cyl column to int
df_18['cyl'] = df_18['cyl'].astype(int)

In [ ]:
df_08.to_csv('data_08.csv', index=False)
df_18.to_csv('data_18.csv', index=False)

In [ ]:
# try using Pandas to_numeric or astype function to convert the
# 2008 air_pollution_score column to float -- this won't work
df_08['air_pollution_score'].nunique()

In [ ]:
df_08['air_pollution_score'].value_counts()

In [ ]:
df_08['air_pollution_score'].astype(float)

# It's not just the air pollution score!
The mpg columns and greenhouse gas scores also seem to have the same problem - maybe that's why these were all saved as strings! According to [this link](http://www.fueleconomy.gov/feg/findacarhelp.shtml#airPollutionScore), which I found from the PDF documentation:

    "If a vehicle can operate on more than one type of fuel, an estimate is provided for each fuel type."
    
Ohh.. so all vehicles with more than one fuel type, or hybrids, like the one above (it uses ethanol AND gas) will have a string that holds two values - one for each. This is a little tricky, so I'm going to show you how to do it with the 2008 dataset, and then you'll try it with the 2018 dataset.

In [ ]:
# First, let's get all the hybrids in 2008
hb_08 = df_08[df_08['fuel'].str.contains('/')]
hb_08

Looks like this dataset only has one! The 2018 has MANY more - but don't worry - the steps I'm taking here will work for that as well!

In [ ]:
hb_18 = df_18[df_18['fuel'].str.contains('/')]
hb_18

We're going to take each hybrid row and split them into two new rows - one with values for the first fuel type (values before the "/"), and the other with values for the second fuel type (values after the "/"). Let's separate them with two dataframes!

In [ ]:
# create two copies of the 2008 hybrids dataframe
df1 = hb_08.copy()  # data on first fuel type of each hybrid vehicle
df2 = hb_08.copy()  # data on second fuel type of each hybrid vehicle

# Each one should look like this
df2

For this next part, we're going use Pandas' apply function. See the docs [here](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.apply.html).

In [ ]:
# columns to split by "/"
split_columns = ['fuel', 'air_pollution_score', 'city_mpg', 'hwy_mpg', 'cmb_mpg', 'greenhouse_gas_score']

# apply split function to each column of each dataframe copy
for c in split_columns:
    df1[c] = df1[c].apply(lambda x: x.split("/")[0])
    df2[c] = df2[c].apply(lambda x: x.split("/")[1])

In [ ]:
# this dataframe holds info for the FIRST fuel type of the hybrid
# aka the values before the "/"s
df1

In [ ]:
# this dataframe holds info for the SECOND fuel type of the hybrid
# aka the values after the "/"s
df2

In [ ]:
# combine dataframes to add to the original dataframe
new_rows = df1.append(df2)

# now we have separate rows for each fuel type of each vehicle!
new_rows

In [ ]:
# drop the original hybrid rows
df_08.drop(hb_08.index, inplace=True)

# add in our newly separated rows
df_08 = df_08.append(new_rows, ignore_index=True)

In [ ]:
# check that all the original hybrid rows with "/"s are gone
df_08[df_08['fuel'].str.contains('/')]

In [ ]:
df_08.shape

# Repeat this process for the 2018 dataset

In [ ]:
# create two copies of the 2018 hybrids dataframe, hb_18
df1 = hb_18.copy()
df2 = hb_18.copy()

### Split values for `fuel`, `city_mpg`, `hwy_mpg`, `cmb_mpg`
You don't need to split for `air_pollution_score` or `greenhouse_gas_score` here because these columns are already ints in the 2018 dataset.

In [ ]:
split_columns = ['fuel','city_mpg','hwy_mpg','cmb_mpg']

# apply split function to each column of each dataframe copy
for c in split_columns:
    df1[c] = df1[c].apply(lambda x : x.split('/')[0])
    df2[c] = df2[c].apply(lambda x : x.split("/")[1])

In [ ]:
# append the two dataframes
new_rows = df1.append(df2)

# drop each hybrid row from the original 2018 dataframe
# do this by using Pandas drop function with hb_18's index
df_18.drop(hb_18.index, inplace=True)

# append new_rows to df_18
df_18 = df_18.append(new_rows,ignore_index = True)

In [ ]:
# check that they're gone
df_18[df_18['fuel'].str.contains('/')]

In [ ]:
df_18.shape

In [ ]:
# convert string to float for 2008 air pollution column
df_08['air_pollution_score'].astype(float)

In [ ]:
# convert int to float for 2018 air pollution column
df_18['air_pollution_score'].astype(float)

In [ ]:
df_08.to_csv('data_08.csv', index=False)
df_18.to_csv('data_18.csv', index=False)

In [3]:
df_08['city_mpg'] = pd.to_numeric(df_08['city_mpg'])

In [13]:
df_08.city_mpg = df_08.city_mpg.astype(float)

In [14]:
df_08.dtypes

model                    object
displ                   float64
cyl                       int64
trans                    object
drive                    object
fuel                     object
veh_class                object
air_pollution_score     float64
city_mpg                float64
hwy_mpg                   int64
cmb_mpg                   int64
greenhouse_gas_score      int64
smartway                 object
dtype: object

In [15]:
# convert mpg columns to floats
mpg_columns = ['city_mpg','hwy_mpg','cmb_mpg']
for c in mpg_columns:
    df_18[c] = df_08[c].astype(float)
    df_08[c] = df_18[c].astype(float)

In [16]:
df_08.dtypes

model                    object
displ                   float64
cyl                       int64
trans                    object
drive                    object
fuel                     object
veh_class                object
air_pollution_score     float64
city_mpg                float64
hwy_mpg                 float64
cmb_mpg                 float64
greenhouse_gas_score      int64
smartway                 object
dtype: object

In [7]:
df_08['greenhouse_gas_score'].dtypes

dtype('int64')

## All the dataypes are now fixed! Take one last check to confirm all the changes.

In [17]:
df_08.dtypes == df_18.dtypes

model                    True
displ                    True
cyl                      True
trans                    True
drive                    True
fuel                     True
veh_class                True
air_pollution_score     False
city_mpg                 True
hwy_mpg                  True
cmb_mpg                  True
greenhouse_gas_score     True
smartway                 True
dtype: bool

In [19]:
# convert string to float for 2008 air pollution column
df_08.air_pollution_score = df_08.air_pollution_score.astype(float)

# convert int to float for 2018 air pollution column
df_18.air_pollution_score = df_18.air_pollution_score.astype(float)

In [20]:
df_08.dtypes == df_18.dtypes

model                   True
displ                   True
cyl                     True
trans                   True
drive                   True
fuel                    True
veh_class               True
air_pollution_score     True
city_mpg                True
hwy_mpg                 True
cmb_mpg                 True
greenhouse_gas_score    True
smartway                True
dtype: bool

In [21]:
# Save your new CLEAN datasets as new files!
df_08.to_csv('clean_08.csv', index=False)
df_18.to_csv('clean_18.csv', index=False)